In [5]:
sub_file = "../data/subs/sub.txt"

subreddits = []
with open(sub_file, "r") as f:
    for line in f:
        subreddits.append(line.strip()[2:])

In [6]:
posts_path = "../data/cleaned_data/"
sql_path = "../../communities-backend/config/populateDB/"
intermeddiate_path = "./intermediateData/"

In [7]:
import json

In [8]:
# load all posts
posts = {}
for sub in subreddits:
    posts[sub] = []
    filename = f"{posts_path}{sub}_posts.json"
    with open(filename, "r") as f:
        posts[sub] = json.load(f)

In [9]:
# load all comments
comments = {}
for sub in subreddits:
    comments[sub] = []
    filename = f"{posts_path}{sub}_comments.json"
    with open(filename, "r") as f:
        comments[sub] = json.load(f)

In [10]:
# Task 1: Filling the posts table

In [12]:
# get all post types

post_types = set()
for sub in subreddits:
    for post in posts[sub]:
        if "post_hint" in post:
            post_types.add(post["post_hint"])

In [13]:
print(post_types)

{'rich:video', 'self', 'hosted:video', 'link', 'image'}


In [83]:
mapping = {'rich:video':'video','link':'link','self':'self','image':'image','hosted:video':'video'}

In [16]:
total = 0
post_count = 0
for sub in subreddits:
    count = 0
    for post in posts[sub]:
        post_count += 1
        if "post_hint" not in post:
            count += 1
    total += count
    print(f"{sub} has {count} posts without post_hint")
print(f"Total number of posts without post_hint: {total}, total posts: {post_count}")

DoesAnybodyElse has 25 posts without post_hint
DoesAnybodyKnow has 0 posts without post_hint
roosterteeth has 18 posts without post_hint
SquaredCircle has 4 posts without post_hint
freegamefindings has 8 posts without post_hint
totalwar has 14 posts without post_hint
de has 14 posts without post_hint
randomacts has 21 posts without post_hint
InteriorDesign has 16 posts without post_hint
socialskills has 24 posts without post_hint
moviescirclejerk has 3 posts without post_hint
networking has 24 posts without post_hint
svenskpolitik has 5 posts without post_hint
grandorder has 15 posts without post_hint
selfimprovement has 25 posts without post_hint
tokyoghoul has 12 posts without post_hint
infographics has 1 posts without post_hint
overwatch has 16 posts without post_hint
legaladvice has 25 posts without post_hint
EDM has 8 posts without post_hint
undertale has 10 posts without post_hint
mechanicadvice has 20 posts without post_hint
needadvice has 25 posts without post_hint
bestoflegala

In [19]:
import os
media_path = "../data/videos/"
thumbnail_path = "../data/video/thumbnails/"
image_path = "../data/images/"

In [21]:
def handle_video(post):
    url = ""
    thumbnail = ""
    post_id = post["id"]
    # check if video file exists in
    filename = f"{media_path}{post_id}.mp4"
    if os.path.exists(filename):
        url = f"videos/{post_id}.mp4"
    filename = f"{thumbnail_path}{post_id}.jpg"
    if os.path.exists(filename):
        thumbnail = f"{post_id}_thumbnail.jpg"
    title = ""
    if "title" in post:
        title = post["title"]
    text = ""
    if "selftext" in post:
        text = post["selftext"]
    
    return title, text, url, thumbnail

In [22]:
def handle_image(post):
    url = ""
    post_id = post["id"]
    # check if video file exists in
    filename = f"{image_path}{post_id}.jpg"
    if os.path.exists(filename):
        url = f"{post_id}.jpg"
    title = ""
    if "title" in post:
        title = post["title"]
    text = ""
    if "selftext" in post:
        text = post["selftext"]
    return title, text, url

In [23]:
def handle_text(post):
    title = ""
    if "title" in post:
        title = post["title"]
    text = ""
    if "selftext" in post:
        text = post["selftext"]
    return title, text

In [24]:
def handle_link(post):
    title = ""
    if "title" in post:
        title = post["title"]
    url = ""
    if "url" in post:
        url = post["url"]
    text =""
    if "selftext" in post:
        text = post["selftext"]
    return title, text, url

In [25]:
def handle_self(post):
    title = ""
    if "title" in post:
        title = post["title"]
    text = ""
    if "selftext" in post:
        text = post["selftext"]
    id = post["id"]
    video_filename = f"{media_path}{id}.mp4"
    image_filename = f"{image_path}{id}.jpg"
    if os.path.exists(video_filename):
        return "video",handle_video(post)
    if os.path.exists(image_filename):
        return "image",handle_video(post)
    return "text",handle_text(post)

In [27]:
def handle_none(post):
    title = ""
    if "title" in post:
        title = post["title"]
    text = ""
    if "selftext" in post:
        text = post["selftext"]
    return title, text

In [28]:
import pandas as pd

df_users = pd.read_csv(f"{intermeddiate_path}username_userid.csv")
df_comm = pd.read_csv(f"{intermeddiate_path}community_id.csv")
df_users.set_index("username", inplace=True)
df_comm.set_index("name", inplace=True)

In [84]:
post_id = 1
post_mapping = {}
count = 0

In [85]:
# all unique subreddits
subs = set()
for sub in subreddits:
    subs.add(sub)

print(len(subs))
print(len(subreddits))

79
79


In [86]:
for sub in subreddits:
    for post in posts[sub]:
        if post["author"] == "[deleted]":
            continue
        count+=1
        if "post_hint" not in post:
            title, text = handle_none(post)
            #replace newline with \\n to avoid sql syntax error
            text = text.replace("'","''")
            title = title.replace("'","''")
            if post["author"] not in df_users.index:
                continue
            user = df_users.loc[post["author"], "id"]
            community = df_comm.loc[sub, "id"]
            command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type) VALUES ('{title}','{text}',{user},{community},'text');"
            command = command.replace("\n","\\n")
            command = command.replace("\t","\\t")
            command = command.replace("\r","\\r")
            with open(f"{sql_path}posts.sql", "a") as f:
                f.write(command+"\n")
        elif mapping[post["post_hint"]] == "video":
            title, text, url, thumbnail = handle_video(post)
            text = text.replace("'","''")
            title = title.replace("'","''")
            user = df_users.loc[post["author"], "id"]
            community = df_comm.loc[sub, "id"]
            command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type,video,image) VALUES ('{title}','{text}',{user},{community},'video','{url}','{thumbnail}');"
            command = command.replace("\n","\\n")
            command = command.replace("\t","\\t")
            command = command.replace("\r","\\r")
            with open(f"{sql_path}posts.sql", "a") as f:
                f.write(command+"\n")
        elif mapping[post["post_hint"]] == "image":
            title, text, url = handle_image(post)
            text = text.replace("'","''")
            title = title.replace("'","''")
            user = df_users.loc[post["author"], "id"]
            community = df_comm.loc[sub, "id"]
            command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type,image) VALUES ('{title}','{text}',{user},{community},'image','{url}');"
            command = command.replace("\n","\\n")
            command = command.replace("\t","\\t")
            command = command.replace("\r","\\r")
            with open(f"{sql_path}posts.sql", "a") as f:
                f.write(command+"\n")
        elif mapping[post["post_hint"]] == "self":
            post_type, (title, text) = handle_self(post)
            user = df_users.loc[post["author"], "id"]
            community = df_comm.loc[sub, "id"]
            text = text.replace("'","''")
            title = title.replace("'","''")
            if post_type == "video":
                command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type,video,image) VALUES ('{title}','{text}',{user},{community},'video','{url}','{thumbnail}');"
            elif post_type == "image":
                command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type,image) VALUES ('{title}','{text}',{user},{community},'image','{url}');"
            else:
                command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type) VALUES ('{title}','{text}',{user},{community},'text');"
            command = command.replace("\n","\\n")
            command = command.replace("\t","\\t")
            command = command.replace("\r","\\r")
            with open(f"{sql_path}posts.sql", "a") as f:
                f.write(command+"\n")
        elif mapping[post["post_hint"]] == "link":
            title, text, url = handle_link(post)
            text = text + f"\n\n[Link]({url})"
            text = text.replace("'","''")
            title = title.replace("'","''")
            user = df_users.loc[post["author"], "id"]
            community = df_comm.loc[sub, "id"]
            command = f"INSERT INTO posts(title,content,creator_id,community_id,post_type) VALUES ('{title}','{text}',{user},{community},'text');"
            command = command.replace("\n","\\n")
            command = command.replace("\t","\\t")
            command = command.replace("\r","\\r")
            with open(f"{sql_path}posts.sql", "a") as f:
                f.write(command+"\n")
        else:
            print(post["post_hint"])
        id = post["id"]
        post_mapping[id] = post_id
        post_id += 1
        

In [87]:
post_id

1933

In [88]:
count

1932

In [93]:
# handling comments
comment_id = 1
to_be_addressed = {}
comment_mapping = {}
for sub in subreddits:
    for comment in comments[sub]:
        parent_id = comment["parent_id"].split("_")[1]
        post_id = comment["link_id"].split("_")[1]
        id = comment["id"]
        body =''
        if post_id not in post_mapping or comment["author"] not in df_users.index:
            continue
        if parent_id == post_id:
            comment_mapping[id] = comment_id
            parent_id = ""
            post_id = post_mapping[post_id]
            body = comment["body"]
        elif parent_id not in comment_mapping:
            # parent comment not yet addressed
            if parent_id not in to_be_addressed:
                to_be_addressed[parent_id] = []
            to_be_addressed[parent_id].append(comment)
            continue
        else:
            comment_mapping[id] = comment_id
            parent_id = f"c_{comment_mapping[parent_id]}"
            post_id = post_mapping[post_id]
            body = comment["body"]
        body = body.replace("'","''")
        comment_id += 1
        command = f"INSERT INTO comments(content,creator_id,post_id,parent_id) VALUES ('{body}','{df_users.loc[comment['author'],'id']}','{post_id}','{parent_id}');"
        command = command.replace("\n","\\n")
        command = command.replace("\t","\\t")
        command = command.replace("\r","\\r")
        with open(f"{sql_path}comments.sql", "a") as f:
            f.write(command+"\n")
        if id in to_be_addressed:
            for c in to_be_addressed[id]:
                comment_mapping[c["id"]] = comment_id
                parent_id = f"c_{comment_mapping[id]}"
                body = c["body"]
                body = body.replace("'","''")
                comment_id += 1
                command = f"INSERT INTO comments(content,creator_id,post_id,parent_id) VALUES ('{body}','{df_users.loc[c['author'],'id']}','{post_id}','{parent_id}');"
                command = command.replace("\n","\\n")
                command = command.replace("\t","\\t")
                command = command.replace("\r","\\r")
                with open(f"{sql_path}comments.sql", "a") as f:
                    f.write(command+"\n")
            del to_be_addressed[id]

In [94]:
# get post with id = 1693 in post_mapping
post = None
for key in post_mapping:
    if post_mapping[key] == 1693:
        post = key
        break

In [95]:
post

'1c6k9yb'

In [96]:
comment_id

635

In [97]:
# count all comments
total = 0
comment_count = 0
for sub in subreddits:
    comment_count += len(comments[sub])
    for comment in comments[sub]:
        if comment["parent_id"].split("_")[1] not in post_mapping:
            total += 1

print(f"Total number of comments: {comment_count}")
print(f"Total number of comments without parent post: {total}")


Total number of comments: 1950
Total number of comments without parent post: 1349


In [99]:
df_id_to_username = pd.read_csv(f"{intermeddiate_path}username_userid.csv")
df_id_to_username.set_index("id", inplace=True)

df_id_to_community = pd.read_csv(f"{intermeddiate_path}community_id.csv")
df_id_to_community.set_index("id", inplace=True)

In [100]:
df_comm_user = pd.read_csv(f"{intermeddiate_path}community_user.csv")

In [103]:
# get members of all communities

community_members = {}
for sub in subreddits:
    community_members[sub] = df_comm_user[df_comm_user["community_id"] == df_comm.loc[sub, "id"]]["user_id"].tolist()

In [105]:
import numpy as np

In [126]:
for sub in subreddits:
    # iterate on each post in the subreddit
    for post in posts[sub]:
        # get bias to +ve votes

        bias = np.random.normal(0.2,0.2)
        if bias >1:
            bias = 1
        if bias < -1:
            bias = -1
        pos_prob = (1 + bias)/2

        # choose percentage of members who will vote
        perc_vote = np.random.normal(0.2,0.1)
        if perc_vote > 1:
            perc_vote = 1
        if perc_vote < 0:
            perc_vote = 0
        
        # randomly choose members who will vote
        members = community_members[sub]
        np.random.shuffle(members)
        num_members = int(len(members)*perc_vote)
        members = members[:num_members]

        # randomly choose members who will vote positively
        np.random.shuffle(members)
        num_pos = int(num_members*pos_prob)
        pos_members = members[:num_pos]
        neg_members = members[num_pos:]

        # get comments on this post
        post_id = post["id"]

        if post_id not in post_mapping:
            continue
        # generate votes
        for mem in pos_members:
            command = f"INSERT INTO votes(vote_type,creator_id,parent_id) VALUES (0,{mem},'p_{post_mapping[post_id]}');"
            with open(f"{sql_path}votes.sql", "a") as f:
                f.write(command+"\n")
        
        for mem in neg_members:
            command = f"INSERT INTO votes(vote_type,creator_id,parent_id) VALUES (-1,{mem},'p_{post_mapping[post_id]}');"
            with open(f"{sql_path}votes.sql", "a") as f:
                f.write(command+"\n")

        # get all comments on this post
        post_comments = []
        for comment in comments[sub]:
            if comment["link_id"].split("_")[1] == post_id and comment["id"] in comment_mapping:
                post_comments.append(comment)
        
        # get bias to +ve votes
        for com in post_comments:
            com_pos_prob = min(max(0,pos_prob + np.random.normal(0,0.1)),1)
            com_perc = min(max(0,perc_vote + np.random.normal(-0.2,0.1)),1)
            np.random.shuffle(members)
            num_members = int(len(members)*com_perc)
            comm_members = members[:num_members]
            np.random.shuffle(comm_members)
            num_pos = int(num_members*com_pos_prob)
            com_pos_members = comm_members[:num_pos]
            com_neg_members = comm_members[num_pos:]

            # get comment id
            if com["id"] not in comment_mapping:
                continue

            com_id = comment_mapping[com["id"]]

            #generate votes
            for mem in com_pos_members:
                command = f"INSERT INTO votes(vote_type,creator_id,parent_id) VALUES (0,{mem},'c_{com_id}');"
                with open(f"{sql_path}votes.sql", "a") as f:
                    f.write(command+"\n")

            for mem in com_neg_members:
                command = f"INSERT INTO votes(vote_type,creator_id,parent_id) VALUES (-1,{mem},'c_{com_id}');"
                with open(f"{sql_path}votes.sql", "a") as f:
                    f.write(command+"\n")

In [134]:
# generate data for moderators

for sub in subreddits:
    if len(community_members[sub]) <= 20:
        continue
    num_mods = np.random.randint(1,5)

    # randomly select num_mods mods
    mods = community_members[sub][:num_mods]
    # print(f"Mods for {sub}: {mods}")

    # privileges: handle-requests, remove-posts, remove-comments, ban-user, pause-comments, remove-post-privilege, remove-comment-privilege, warning
    '''
    1: handle-requests
    2: ban-user
    3: remove-post-privilege
    4: remove-comment-privilege
    5: remove-posts
    6: remove-comments
    7: pause-comments
    8: warning
    '''

    req_prev = 0
    ban_prev = 0
    rem_post_prev = 0
    rem_comm_prev = 0
    rem_post_priv_prev = 0
    rem_comm_priv_prev = 0
    pause_comm_prev = 0
    warning_prev = 1
    choice = np.random.rand()
    if choice>0.7:
        req_prev = 1
    choice = np.random.rand()
    if choice>0.6:
        ban_prev = 1
    choice = np.random.rand()
    if choice>0.55:
        rem_post_priv_prev = 1
    choice = np.random.rand()
    if choice>0.5:
        rem_comm_priv_prev = 1
    choice = np.random.rand()
    if choice>0.4:
        rem_post_prev = 1
    choice = np.random.rand()
    if choice>0.35:
        rem_comm_prev = 1
    choice = np.random.rand()
    if choice>0.3:
        pause_comm_prev = 1
    choice = np.random.rand()
    if choice>0.2:
        warning_prev = 1

    command = f"INSERT INTO moderators(user_id, community_id, privileges) VALUES({mods[0]},{df_comm.loc[sub,'id']},'{req_prev}{ban_prev}{rem_post_priv_prev}{rem_comm_priv_prev}{rem_post_prev}{rem_comm_prev}{pause_comm_prev}{warning_prev}');"
    with open(f"{sql_path}moderators.sql", "a") as f:
        f.write(command+"\n")

In [144]:
import copy
# get a list of all user_ids
user_ids = df_id_to_username.index.tolist()
# create a deep copy of users_id
temp_user_ids = copy.deepcopy(user_ids)
print(len(user_ids))
print(len(temp_user_ids))

3058
3058


In [147]:
# follower
for follower in user_ids:
    perc = max(min(np.random.normal(0,0.01)+0.001,1),0)
    np.random.shuffle(temp_user_ids)
    num_followers = int(len(temp_user_ids)*perc)
    print(num_followers)
    followed = temp_user_ids[:num_followers]
    for followee in followed:
        command = f"INSERT INTO followers(follower_id, followed_id) VALUES({follower},{followee});"
        with open(f"{sql_path}followers.sql", "a") as f:
            f.write(command+"\n")

11
0
0
1
0
0
0
28
5
0
0
0
25
0
11
7
0
0
0
0
0
0
3
15
0
0
29
0
0
9
37
0
17
8
3
12
12
12
0
23
0
39
18
0
0
0
0
0
0
0
0
23
0
0
0
53
31
23
0
5
10
36
0
5
0
44
0
15
0
17
0
0
56
0
0
25
0
17
38
31
0
13
20
14
0
0
0
0
0
40
0
0
19
0
0
1
39
31
0
0
12
7
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
35
0
14
0
9
0
0
21
33
10
0
46
35
0
4
0
0
0
33
11
0
24
0
0
0
0
5
0
23
0
0
46
19
8
0
5
20
42
23
4
17
0
5
0
35
0
2
0
28
0
17
0
0
0
1
0
0
0
11
33
12
7
0
0
0
0
0
46
28
3
25
65
0
44
30
17
0
5
21
0
0
17
0
3
0
12
28
0
71
0
0
0
27
0
23
0
0
22
0
34
0
0
17
0
0
0
55
48
1
0
0
0
13
0
12
25
9
3
57
0
0
1
50
0
11
59
0
6
0
45
0
0
3
16
0
12
25
0
0
0
6
47
1
25
0
28
0
0
0
0
0
0
0
96
0
27
11
36
0
32
50
7
33
0
3
26
26
0
53
0
0
16
46
11
0
0
0
0
0
33
0
13
0
47
6
48
27
5
51
0
32
12
0
0
13
0
0
32
9
0
0
2
0
0
5
41
45
14
4
0
0
0
0
53
7
7
0
0
4
52
55
45
52
32
0
0
0
0
0
0
27
0
0
29
0
0
0
18
17
57
0
0
0
0
1
0
41
0
13
0
0
7
0
0
0
0
0
42
0
0
0
39
0
0
0
0
10
46
10
48
13
9
6
0
0
7
46
52
38
11
0
87
0
0
0
0
1
0
0
0
0
24
0
0
0
16
82
0
10
35
41
0
25
16
0
0
1

In [148]:
# blocked table
for blocker in user_ids:
    perc = min(max(np.random.normal(0,0.01) - 0.02,0),1)
    if perc==0:
        continue
    np.random.shuffle(temp_user_ids)
    num_blocked = int(len(temp_user_ids)*perc)
    blocked = temp_user_ids[:num_blocked]
    for block in blocked:
        command = f"INSERT INTO blocked(blocker_id, blocked_id) VALUES({blocker},{block});\n"
        with open(f"{sql_path}blocked.sql", "a") as f:
            f.write(command)